<a href="https://colab.research.google.com/github/Nikhil1601/Data-analysis-using-gemini-LLM/blob/main/gemini_llm_da.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain

!pip install langchain_experimental
!pip install -U --quiet langchain-google-genai langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
df = pd.read_csv('/content/olympics-economics.csv')
df.head()

,country,country_code,gold,silver,bronze,total,gdp,gdp_year,population
0,United States,USA,40,44,42,126,81695.19,2023,334.9
1,China,CHN,40,27,24,91,12614.06,2023,1410.7
2,Japan,JPN,20,12,13,45,33834.39,2023,124.5
3,Australia,AUS,18,19,16,53,64711.77,2023,26.6
4,France,FRA,16,26,22,64,44460.82,2023,68.2


In [ ]:
df.sort_values(by='silver', ascending=False).head(1)

,country,country_code,gold,silver,bronze,total,gdp,gdp_year,population
0,United States,USA,40,44,42,126,81695.19,2023,334.9


In [ ]:
import os

os.environ["GOOGLE_API_KEY"] = ""

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
# llm = ChatOpenAI(temperature=0.5)
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.5,
                             google_api_key=os.environ["GOOGLE_API_KEY"])

In [ ]:
# df = pd.read_csv('/content/newdata (1).csv')
# df.head()

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True,
    allow_dangerous_code=True
)

In [ ]:
agent.invoke("Which circle have the highest number of calls?")



> Entering new AgentExecutor chain...
Thought: I should find the unique values of the CIRCLE column, and then count the number of occurrences of each value. The value with the highest count is the circle with the highest number of calls.
Action: python_repl_ast
Action Input: df['CIRCLE'].value_counts().idxmax()DelhiFinal Answer: Delhi

> Finished chain.


{'input': 'Which circle have the highest number of calls?', 'output': 'Delhi'}

In [ ]:
agent.run("Who got max medals with population between 44.5 and 49.8?")

<ipython-input-8-af8812fe7b00>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("Who got max medals with population between 44.5 and 49.8?")




> Entering new AgentExecutor chain...
Thought: I can use the `query` function to filter the dataframe by population, and then use the `idxmax` function to find the row with the maximum total medals.
Action: python_repl_ast
Action Input: df.query('population > 44.5 and population < 49.8').idxmax()['total']14Final Answer: 14

> Finished chain.


'14'

In [ ]:
query = "Who got max medals with gdp between 22147 and 48527?"
agent.invoke(query)



> Entering new AgentExecutor chain...
Thought: I should filter the dataframe by the gdp range and then find the country with the maximum total medals.
Action: python_repl_ast
Action Input: df[(df['gdp'] >= 22147) & (df['gdp'] <= 48527)]           country country_code  gold  silver  bronze  total       gdp  \
2            Japan          JPN    20      12      13     45  33834.39   
4           France          FRA    16      26      22     64  44460.82   
7      South Korea          KOR    13       9      10     32  33121.37   
8            Italy          ITA    12      13      15     40  38373.17   
13         Hungary          HUN     6       7       6     19  22147.21   
14           Spain          ESP     5       4       9     18  32676.98   
27  Czech Republic          CZE     3       0       2      5  30427.42   
32         Bahrain          BHR     2       1       1      4  29084.31   
33        Slovenia          SVN     2       1       0      3  32163.51   
34          Taiwan    

{'input': 'Who got max medals with gdp between 22147 and 48527?',
 'output': 'France'}

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_csv_agent
agent_executer = create_csv_agent(llm, '/content/olympics-economics.csv',
                                   verbose=True,allow_dangerous_code=True)

In [ ]:
agent_executer.invoke("Which country has the highest number of silver medals?")



> Entering new AgentExecutor chain...
Thought: I should sort the dataframe by the silver medal count and then get the first row.
Action: python_repl_ast
Action Input: df.sort_values("silver", ascending=False).head(1)         country country_code  gold  silver  bronze  total       gdp  \
0  United States          USA    40      44      42    126  81695.19   

   gdp_year  population  
0      2023       334.9  The United States has the highest number of silver medals.
Final Answer: United States

> Finished chain.


{'input': 'Which country has the highest number of silver medals?',
 'output': 'United States'}

In [ ]:
agent_executer.invoke("Who got max medals with population between 44.5 and 49.8?")



> Entering new AgentExecutor chain...
Thought: I need to first filter the dataframe to only include countries with a population between 44.5 and 49.8. Then, I can find the country with the maximum total medals.
Action: python_repl_ast
Action Input: df[(df['population'] > 44.5) & (df['population'] < 49.8)]      country country_code  gold  silver  bronze  total       gdp  gdp_year  \
14      Spain          ESP     5       4       9     18  32676.98      2023   
38    Algeria          DZA     2       0       1      3   5260.21      2023   
51  Argentina          ARG     1       1       1      3  13730.51      2023   
57     Uganda          UGA     1       1       0      2   1014.21      2023   

    population  
14        48.4  
38        45.6  
51        46.7  
57        48.6  Now, I can find the country with the maximum total medals.
Action: python_repl_ast
Action Input: df[(df['population'] > 44.5) & (df['population'] < 49.8)].sort_values(by='total', ascending=False).head(1)   countr

{'input': 'Who got max medals with population between 44.5 and 49.8?',
 'output': 'Spain'}

#Checking approach with llama


In [ ]:
!pip install langchain_experimental
!pip install langchain
!pip install transformers

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [ ]:
!pip install langchain_community

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
import pandas as pd

# Load the Llama 2 model and tokenizer
model_name = "meta-llama/Llama-2-7b-hf"  # Replace with the desired Llama 2 model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# tokenizer.model_max_length = 1024
# Create the Hugging Face Pipeline for text generation
generate_text = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=521
)


In [ ]:
hf = HuggingFacePipeline(pipeline=generate_text)

# Load your CSV data into a pandas DataFrame
df = pd.read_csv("/content/olympics-economics.csv")

# Create the CSV agent using the Hugging Face Pipeline
agent = create_pandas_dataframe_agent(
    hf,
    df,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    allow_dangerous_code=True
)

# Interact with the agent
query = "Which country has highest silver medals?"
print(agent.run(query))

<ipython-input-6-144768ddcbeb>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hf = HuggingFacePipeline(pipeline=generate_text)
<ipython-input-6-144768ddcbeb>:17: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(agent.run(query))
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generat



> Entering new AgentExecutor chain...


Empty: 